In [3]:
from rerankers import Reranker
from rich import print

In [41]:
# ranker = Reranker('cross-encoder')
# ranker = Reranker('avsolatorio/GIST-large-Embedding-v0')
# ranker = Reranker('flashrank')
# ranker = Reranker('ce-esci-MiniLM-L12-v2', model_type='flashrank')
# ranker = Reranker("t5")
# ranker = Reranker("unicamp-dl/InRanker-base", model_type = "t5")
# ranker = Reranker("colbert")
ranker = Reranker("mixedbread-ai/mxbai-rerank-large-v1", model_type="cross-encoder")

Loading TransformerRanker model mixedbread-ai/mxbai-rerank-large-v1
No device set
Using device cuda
No dtype set
Using dtype torch.float16


config.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

Loaded model mixedbread-ai/mxbai-rerank-large-v1
Using device cuda.
Using dtype torch.float16.


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

In [42]:
texts = [
    "I like to play soccer",
    "I like to play football",
    "I like to play basketball",
    "I love dogs",
    "Catcher in the Rye is a great book",
]

In [43]:
results = ranker.rank(query="What's your favorite sport?", docs=texts)

print(results)

RankedResults(
    results=[
        Result(doc_id=0, text='I like to play soccer', score=-1.2607421875, rank=1),
        Result(doc_id=2, text='I like to play basketball', score=-1.2890625, rank=2),
        Result(doc_id=1, text='I like to play football', score=-1.9384765625, rank=3),
        Result(doc_id=3, text='I love dogs', score=-5.12109375, rank=4),
        Result(doc_id=4, text='Catcher in the Rye is a great book', score=-6.19140625, rank=5)
    ],
    query="What's your favorite sport?",
    has_scores=True
)

In [44]:
print([document.text for document in results.results])

[
    'I like to play soccer',
    'I like to play basketball',
    'I like to play football',
    'I love dogs',
    'Catcher in the Rye is a great book'
]

# Testing `query_similar_docs`

In [46]:
embedded_question = get_embeddings(question)
    db_conn = get_db_conn()
    num_docs = 20 if use_reranking else 5
    # get (content, url) tuples for the top n similar documents
    top_similar_docs = get_topn_similar_docs(
        embedded_question, db_conn, n=num_docs, include_metadata=True
    )

    if use_reranking:
        urls = rerank_documents(question, top_similar_docs)[:5]
    else:
        urls = [doc[1] for doc in top_similar_docs]  # Unpacking URLs

    return (question, url_ending, urls)


ImportError: attempted relative import with no known parent package

# Simplistic TF-IDF Reranker

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Toy document corpus
documents = [
    "The quick brown fox jumps over the lazy dog",
    "A quick brown fox jumps over the lazy dog",
    "The quick brown fox is quick and brown",
    "The lazy dog is lazy and sleepy",
    "The quick brown fox is different from the lazy dog",
]

# Toy queries and their corresponding relevant document indices
queries = [("quick fox", [0, 1, 2]), ("lazy dog", [3, 4]), ("brown fox", [0, 1, 2, 4])]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Vectorize documents
document_vectors = vectorizer.fit_transform(documents)

# Train a logistic regression reranker
reranker = LogisticRegression(random_state=42)
X_train = []
y_train = []

for query, relevant_docs in queries:
    query_vector = vectorizer.transform([query])
    for doc_idx, doc_vector in enumerate(document_vectors):
        X_train.append(
            np.concatenate((query_vector.toarray()[0], doc_vector.toarray()[0]))
        )
        y_train.append(1 if doc_idx in relevant_docs else 0)

reranker.fit(X_train, y_train)

# Example usage of the trained reranker
query = "quick brown fox"
query_vector = vectorizer.transform([query])
scores = []

for doc_vector in document_vectors:
    input_vector = np.concatenate((query_vector.toarray()[0], doc_vector.toarray()[0]))
    score = reranker.predict_proba([input_vector])[0][1]
    scores.append(score)

ranked_indices = np.argsort(scores)[::-1]
print("Ranked documents for query:", query)
for idx in ranked_indices:
    print("Document:", documents[idx])
    print("Score:", scores[idx])
    print()

Ranked documents for query: quick brown fox
Document: A quick brown fox jumps over the lazy dog
Score: 0.6937165451385258

Document: The quick brown fox jumps over the lazy dog
Score: 0.6928630071635998

Document: The quick brown fox is quick and brown
Score: 0.6868308019742143

Document: The quick brown fox is different from the lazy dog
Score: 0.6802242759508812

Document: The lazy dog is lazy and sleepy
Score: 0.5727275080137214



In [5]:
from rerankers import Reranker

ranker = Reranker("cross-encoder")

texts = [
    "I like to play soccer",
    "I like to play football",
    "War and Peace is a great book",
    "I love dogs",
    "Ginger cats aren't very smart",
    "I like to play basketball",
]

results = ranker.rank(query="What's your favorite sport?", docs=texts)

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Default Model: mixedbread-ai/mxbai-rerank-base-v1
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1
No device set
Using device cuda
No dtype set
Using dtype torch.float16
Loaded model mixedbread-ai/mxbai-rerank-base-v1
Using device cuda.
Using dtype torch.float16.


In [6]:
print(results)

RankedResults(
    results=[
        Result(doc_id=5, text='I like to play basketball', score=-0.46533203125, rank=1),
        Result(doc_id=0, text='I like to play soccer', score=-0.7353515625, rank=2),
        Result(doc_id=1, text='I like to play football', score=-0.9677734375, rank=3),
        Result(doc_id=2, text='War and Peace is a great book', score=-5.40234375, rank=4),
        Result(doc_id=3, text='I love dogs', score=-5.5859375, rank=5),
        Result(doc_id=4, text="Ginger cats aren't very smart", score=-5.94921875, rank=6)
    ],
    query="What's your favorite sport?",
    has_scores=True
)